In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data
past_2_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W29`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W30`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10) # set query limit to 10G
query_job = bigquery_client.query(past_2_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
df_2930 = rows.to_dataframe()

In [4]:
df_2930 ## week 26 to week 30

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
0,2020-07-11 15:19:00+00:00,2020-07-11 15:29:00+00:00,Break,None,B4,rework previous mks,None,KL1,None,Saturday Afternoon,Melanie Dua;,3,10.70,2020-29
1,2020-07-13 13:55:00+00:00,2020-07-13 15:52:00+00:00,Break,None,B4,clean up,None,KL1,None,Monday Afternoon,iXUsr_Chan(8814),3,116.87,2020-29
2,2020-07-12 21:49:00+00:00,2020-07-12 22:06:00+00:00,Break,None,B4,clean up,None,KL1,None,Sunday Afternoon,iXUsr_Chang(8908),3,17.42,2020-29
3,2020-07-10 12:58:00+00:00,2020-07-10 13:07:00+00:00,Break,None,A4,Unselected,None,KL2,None,Friday Morning,Eve Yerafael;,4,9.75,2020-29
4,2020-07-10 14:37:00+00:00,2020-07-10 14:48:00+00:00,Break,None,A4,Unselected,None,KL2,None,Friday Afternoon,Eve Yerafael;,4,10.98,2020-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12905,2020-07-17 19:29:41+00:00,2020-07-17 22:08:49+00:00,Break,None,SF,move to other line,None,KL20,None,Friday Afternoon,Dummy;,3,159.13,2020-30
12906,2020-07-17 17:35:50+00:00,2020-07-17 18:14:44+00:00,Break,None,B4,rework previous mks,None,KL4,None,Friday Afternoon,Melanie Dua;,4,38.90,2020-30
12907,2020-07-18 20:07:21+00:00,2020-07-18 20:19:45+00:00,Break,None,G4,rework previous mks,None,KL9,None,Saturday Afternoon,iXUsr_Hular(8441),3,12.40,2020-30
12908,2020-07-18 15:34:50+00:00,2020-07-18 15:43:01+00:00,Break,None,B2,rework previous mks,None,KL13,None,Saturday Afternoon,Melanie Dua;,4,8.18,2020-30


In [9]:
import datetime as dt
import pandas as pd

In [33]:
# actual_start_time = test_group.sort_values(["Finish_Time"]).iloc[0]['Finish_Time'].tz_localize(None)
# current_date = dt.datetime.date(actual_start_time)
# morning_time = dt.time(6, 5) # morning shift suppose to start at 6:05 
# shift_start_time = dt.datetime.combine(current_date, morning_time)

In [42]:
# (actual_start_time - shift_start_time).seconds / 60

9.0

In [19]:
# index_and_sort = test_group.set_index('Finish_Time').sort_index()
# current_date = dt.datetime.date(index_and_sort.index[0])
# morning_time = dt.time(14, 0)
# shift_start_time = dt.datetime.combine(current_date, morning_time)
# print(shift_start_time)
# index_and_sort.index.get_loc(shift_start_time, method='nearest')

In [58]:
def find_start_time(grouped_df):
    week_day = grouped_df['Event_Shift'].str.split().iloc[0][0]
    shift = grouped_df['Event_Shift'].str.split().iloc[0][1]
    first_event = grouped_df.sort_values(["Finish_Time"]).iloc[0]
    actual_start_time = first_event['Finish_Time'].tz_localize(None)
    current_date = dt.datetime.date(actual_start_time)
    morning_time = dt.time(6, 5) # morning shift suppose to start at 6:05 
    afternoon_time = dt.time(14, 20)
    if week_day in ['Thursday', 'Friday']:
        if (shift == "Afternoon"):
            shift_start_time = dt.datetime.combine(current_date, afternoon_time)
            lateness = (actual_start_time - shift_start_time).seconds / 60

        elif (shift == "Morning"):
            shift_start_time = dt.datetime.combine(current_date, morning_time)
            lateness = (actual_start_time - shift_start_time).seconds / 60

        else:
            print("Error")
    else:
        shift_start_time = dt.datetime.combine(current_date, morning_time)
        lateness = (actual_start_time - shift_start_time).seconds / 60

    return (lateness if (lateness < 30) else 'not on kitting at the production begain')


In [66]:
df_lateness = pd.DataFrame( {'lateness': df_2930.groupby(["Team_Leader",  "Week", "Event_Shift"]).apply(find_start_time)})

In [67]:
df_lateness

lateness
Team_Leader           Week    Event_Shift                                                
Bayu Agus Alfian      2020-29 Monday Afternoon    not on kitting at the production begain
                              Monday Morning                                            9
                              Saturday Morning                                          5
                              Sunday Morning                                           12
                              Thursday Morning                                         20
...                                                                                   ...
iXUsr_Ushida(8909)    2020-30 Saturday Afternoon  not on kitting at the production begain
iXUsr_Wilson(8678)    2020-29 Friday Afternoon                                         23
iXUsr_Ye'Rafael(8520) 2020-29 Monday Afternoon    not on kitting at the production begain
                      2020-30 Monday Morning      not on kitting at the production begain
                              Thursday Morning    not on kitting at the production begain

[197 rows x 1 columns]

In [ ]:
df_lateness.to_csv